#### 1. 라이브러리 선언

In [4]:
import pandas as pd
import polars as pl
import datashader as ds
from datashader import transfer_functions as tf
import plotly.io as pio

import numpy as np

#### 2. 데이터 로드

In [7]:

# CSV 파일 읽기, 필요한 컬럼 선택
dong_list = ["송도동", "청라동", "중산동"]
file_path = '.'

file_name = "utf_인천광역시_온도_202107.csv" # 파일 이름은 여기 수정하면됨

df = pl.read_csv(f'{file_path}/{file_name}')

dff = df.select(['위도', '경도', '온도']).to_pandas()


# Datashader Canvas 설정
cvs = ds.Canvas(plot_width=3200, plot_height=4800)

# 포인트를 그리고, '온도' 컬럼을 사용하여 색상 지정
agg = cvs.points(dff, x='경도', y='위도', agg=ds.mean('온도'))

v = agg.values

vmin = np.nanmin(v)
vmax = np.nanmax(v)

print("최저 평균 온도 : ", vmin)
print("최고 평균 온도 : ", vmax)

# agg is an xarray object, see http://xarray.pydata.org/en/stable/ for more details
coords_lat, coords_lon = agg.coords['위도'].values, agg.coords['경도'].values

# Corners of the image, which need to be passed to mapbox
coordinates = [[coords_lon[0], coords_lat[0]],
               [coords_lon[-1], coords_lat[0]],
               [coords_lon[-1], coords_lat[-1]],
               [coords_lon[0], coords_lat[-1]]]
# print(coordinates)

df # 데이터 출력

최저 평균 온도 :  24.397
최고 평균 온도 :  49.855


위도,경도,시간,동,온도
f64,f64,str,str,f64
37.420721,126.637859,"""2021-07-02 22:12:44""","""송도동""",34.221
37.420549,126.637859,"""2021-07-02 22:12:45""","""송도동""",34.221
37.420378,126.637859,"""2021-07-02 22:12:46""","""송도동""",34.247
37.420206,126.637859,"""2021-07-02 22:12:47""","""송도동""",34.247
37.420034,126.637859,"""2021-07-02 22:12:48""","""송도동""",34.234
…,…,…,…,…
37.529898,126.660519,"""2021-07-19 14:06:52""","""청라동""",39.778
37.529898,126.660862,"""2021-07-19 14:06:53""","""청라동""",39.778
37.529898,126.660862,"""2021-07-19 14:06:54""","""청라동""",39.791


#### 3. 이미지 생성

In [8]:

from colorcet import fire
import datashader.transfer_functions as tf
import matplotlib.cm as cm

# 점 크기 변경
agg = tf.spread(agg, px=3)
# img = tf.shade(agg, cmap=['blue', 'green', 'red'], how='linear')[::-1].to_pil()
img = tf.shade(agg, cmap=cm.coolwarm, alpha=180)[::-1].to_pil()

# Pillow를 사용하여 블러 필터 적용
from PIL import ImageFilter
blurred_img = img.filter(ImageFilter.GaussianBlur(radius=1))
# blurred_img.show()

#### 4. 지도 + 이미지

In [10]:

import plotly.express as px
# Trick to create rapidly a figure with mapbox axes
fig = px.scatter_mapbox(dff[:1], lat='위도', lon='경도', zoom=12, opacity=1, height=1200) # height 바꾸면 맵 이미지 바뀜 이거 바꿔야함
# Add the datashader image as a mapbox layer image
fig.update_layout(mapbox_style="carto-positron",
                 mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": blurred_img,
                    "coordinates": coordinates
                }]
)


# 색상 맵 만들기
# import plotly.graph_objects as go
# colorscale = cm.coolwarm(np.linspace(0, 1, 256))
# colorscale = [[i/255, f'rgb({int(colorscale[i][0]*255)}, {int(colorscale[i][1]*255)}, {int(colorscale[i][2]*255)})'] for i in range(256)]

# fig.update_layout(coloraxis=dict(colorscale=colorscale, cmin=vmin, cmax=vmax))

# 빈 scatter trace 추가하여 색상바를 활성화
# fig.add_trace(go.Scattermapbox(
#     lat=[None],
#     lon=[None],
#     name='MYTRACE',
#     mode='markers',
#     marker=dict(
#         showscale=True,
#         cmin=vmin, # 최소
#         cmax=vmax, # 최대
#         colorscale=colorscale,
#         colorbar=dict(
#             title="평균온도",
#             titleside="top",
#             x=0.6,  # x 위치 (0 ~ 1 사이 값)
#             y=0.97,  # y 위치 (0 ~ 1 사이 값)
#             xanchor='left',  # x축 기준 위치
#             yanchor='top',  # y축 기준 위치
#             orientation='h',
#             len=0.4
#         )
#     )
# ))

fig.show()
html_content = fig.to_html()
# fig.write_image('./test.jpg')
# import matplotlib.pyplot as plt
# plt.savefig('./test.jpg')